In [41]:
from os import listdir
from os.path import isfile, join
import os
import numpy as np
from amrafile import amrafile as af
from amracommon.analysis.registration import normalized_cross_correlation
from scipy.ndimage import gaussian_filter, sobel
import matplotlib.pyplot as plt

testTargets =  ["0013B","0013C","0013D","0013E","0013F","00130","00131",
     "00132","00133","00134","00135","00136","00137","0014A","0014B",
    "0014D","0014F","00140","00141","00142","00143","00144","00145",
     "00146","00147","0015A","0015B","0015C","0015D","0015E","0015F"]

#testTargets = ["00040","00041","00042","00043","00044","00045","00046","00047",
#               "00048","00034","00035","00036","00037","00038","00039","0003A","0003B",
#               "0003C","0001A","0001B","0001C","0001D","0001E","0001F","00020","00021","00022","00023",
#                "00024","00025","0003D"]

searchSize = [22, 22, 22]

error = []

testPoi = 'T9'

sortedDist = np.zeros((len(testTargets),29))
sortedNcc = sortedDist.copy()

    
for k, target in enumerate(testTargets):

    signal = af.parse('/moria/data/DB/0030/'+target+'/wholebody_normalized_water_1_'+target+'.amra')
    prototypePath = '/media/hannes/localDrive/DB/0030/'+target+'/prototypes'
    
    prototypes = [join(prototypePath,f) for f in listdir(prototypePath)  if isfile(join(prototypePath, f))]
    
    nbrOfPrototypes = len(prototypes)
    
    targetPoi = signal.get_poi(testPoi)
    voxelSize = signal.voxel_size()
    
    reducedSize = np.round(searchSize/np.array(voxelSize)).astype(int)
    
    sobelPrototype = np.zeros((reducedSize*2+1))
    sobelTarget = sobelPrototype.copy()
        
    prototypeSignals = [af.parse(prototype) for prototype in prototypes]
    
    prototypePois = [sig.get_poi(testPoi) for sig in prototypeSignals]
        
    nccLocal, nccFull = [], []
    
    targetData = signal.data
    
    reducedPrototypes, reducedTargets = [], []

    for ind, poi in enumerate(prototypePois):

        z_lower = poi[0]-reducedSize[0]
        z_upper = poi[0]+reducedSize[0]+1
        y_lower = poi[1]-reducedSize[1]
        y_upper = poi[1]+reducedSize[1]+1
        x_lower = poi[2]-reducedSize[2]
        x_upper = poi[2]+reducedSize[2]+1

        prototype = prototypeSignals[ind].data

        ''' Extract reduced space from prototype and target'''
        reducedPrototype = prototype[z_lower:z_upper, y_lower:y_upper, x_lower:x_upper]
        reducedTarget = targetData[z_lower:z_upper, y_lower:y_upper, x_lower:x_upper]
        
        #reducedPrototype = prototype[z_lower:z_upper,:,:]
        #reducedTarget = targetData[z_lower:z_upper,:,:]
        
        reducedPrototype = gaussian_filter(reducedPrototype,0.5,mode='constant')
        reducedTarget = gaussian_filter(reducedTarget,0.5,mode='constant')

        ''' Calculate ncc and store in lists'''
        nccFull.append(normalized_cross_correlation(prototype[0:prototype.shape[0]-5, 0:prototype.shape[1]-5,
                                                              0:prototype.shape[2]-5], targetData[0:prototype.shape[0]-5,
                                                              0:prototype.shape[1]-5, 0:prototype.shape[2]-5]))
        
        nccLocal.append(normalized_cross_correlation(reducedPrototype, reducedTarget))
        
        reducedPrototypes.append(reducedPrototype)
        reducedTargets.append(reducedTarget)
    
    ncc = list(np.array(nccLocal) + np.array(nccFull))
    poiIndex = ncc.index(max(ncc))
    worstIndex = ncc.index(min(ncc))
    
    #plt.figure(frameon =False)
    #currentAxis = plt.gca()
    #plt.imshow((reducedPrototypes[poiIndex][30,:,:]),  plt.get_cmap('gray'), interpolation = 'nearest', origin='lower')
    #plt.autoscale(False)
    #plt.colorbar()

    #plt.figure(frameon =False)
    #currentAxis = plt.gca()
    #plt.imshow((reducedTargets[poiIndex][30,:,:]),  plt.get_cmap('gray'), interpolation = 'nearest', origin='lower')
    #plt.autoscale(False)
    #plt.colorbar()
    
    #plt.figure(frameon =False)
    #currentAxis = plt.gca()
    #plt.imshow((reducedPrototypes[worstIndex][30,:,:]),  plt.get_cmap('gray'), interpolation = 'nearest', origin='lower')
    #plt.autoscale(False)
    #plt.colorbar()

    #plt.figure(frameon =False)
    #currentAxis = plt.gca()
    #plt.imshow((reducedTargets[worstIndex][30,:,:]),  plt.get_cmap('gray'), interpolation = 'nearest', origin='lower')
    #plt.autoscale(False)
    #plt.colorbar()


    
    ''' Repmat the target poi to compare distances'''
    repTargetPoi = np.tile(targetPoi, (nbrOfPrototypes,1))
    repTargetSize = np.tile(voxelSize, (nbrOfPrototypes,1))

    ''' Diff between target poi ground truth and every deformed prototype poi in mm'''
    poiDiff = list(np.sqrt(np.sum(((prototypePois - repTargetPoi)*voxelSize)**2,1)))
    
    zDiff = list((abs(prototypePois - repTargetPoi))[:,0])
    
    diffList = np.array(list(zip(poiDiff,ncc)))
    
    bestIndex = zDiff.index(min(zDiff))
    
    #plt.figure(frameon =False)
    #currentAxis = plt.gca()
    #plt.imshow((reducedPrototypes[bestIndex][30,:,:]),  plt.get_cmap('gray'), interpolation = 'nearest', origin='lower')
    #plt.autoscale(False)
    #plt.colorbar()

    #plt.figure(frameon =False)
    #currentAxis = plt.gca()
    #plt.imshow((reducedTargets[bestIndex][30,:,:]),  plt.get_cmap('gray'), interpolation = 'nearest', origin='lower')
    #plt.autoscale(False)
    #plt.colorbar()
    


    sortedList = np.array([(x,y) for (y,x) in sorted(zip(ncc,zDiff))])
    
    sortedDist[k,:] = sortedList[:,0]
    sortedNcc[k,:] = sortedList[:,1]

    nccDiff = poiDiff[poiIndex]
    error.append(zDiff)
    
    #print(zDiff[poiIndex])
    #print(zDiff[worstIndex])
    #print(zDiff[bestIndex])

    ''' Differences sorted in ascending order'''
    #sorted_diff = sorted(poi_diff)

    ''' Best poi to choose from prototypes '''
    #prototype_poi_index = poi_diff.index(min(poi_diff))
    plt.show()
        
    print(k)

meanDist = np.mean(sortedDist,0)
stdNcc = np.std(sortedNcc,0)
meanNcc = np.mean(sortedNcc,0)
stdDist = np.std(sortedDist,0)

plt.figure(frameon =False)
currentAxis = plt.gca()
plt.plot(range(0,len(prototypes)),meanDist)
plt.fill_between(range(0,len(prototypes)), meanDist-stdDist, meanDist+stdDist, alpha=0.3, edgecolor='Blue', facecolor='Blue', linewidth = 0)
plt.autoscale(False)
plt.show()


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30


In [44]:
nccDiff

8.70414851855171

In [7]:
plt.figure(frameon =False)
currentAxis = plt.gca()
plt.plot(range(0,len(prototypes)),meanDist)
plt.fill_between(range(0,len(prototypes)), meanDist-stdDist, meanDist+stdDist, alpha=0.3, edgecolor='Blue', facecolor='Blue', linewidth = 0)
plt.autoscale(False)
plt.ylim(0,15)
#plt.savefig('poiInit_T9_z.png', bbox_inches='tight')
plt.show()

In [126]:
meanDist /= np.amax(meanDist)

In [140]:
nbrOfPrototypes

29

In [32]:
np.mean(zDiff)*voxelSize[0]

15.702831511073732

In [34]:
a = [1,2,3]
b = [3,2,1]

In [36]:
np.array(a)+np.array(b)

array([4, 4, 4])

In [169]:
target_poi = (100,50,50)
target_voxel_size = (3,2.23,2.23)

mean_dev = [ 1,  2,  2]
mean_std = np.array([ 1,  2,  2])

''' Directional combinations'''
dir_comb = np.array([[1,1,1],[1,1,-1],[1,-1,-1],[-1,1,-1],[1,-1,1],[-1,-1,-1]])

''' Take directions into account in the mean POI deviation'''
mean_temp = dir_comb[np.random.randint(0,len(dir_comb))]*mean_dev

''' Add positional noise to POI position'''
poi = target_poi + np.round([np.random.normal(0, y) + x for x,y in zip(mean_temp, mean_std)]).astype(int)

print(poi)

[98 47 47]


In [160]:
np.sqrt(sum(((poi - target_poi)*target_voxel_size)**2))

4.3526773369961616

In [121]:
dir_comb[np.random.randint(0,len(dir_comb))]*mean_dev/target_voxel_size

array([ 0.4017094 ,  0.88536277,  0.56341267])